# 한글 토크나이징 with konlpy

[출처 : 오늘코드](https://github.com/corazzon/KaggleStruggle/)

In [ ]:
# jdk JVMNotFoundException (https://github.com/konlpy/konlpy/issues/24 
# jdk 설치 https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html
# .net framework 설치 : OS 맞게 설치필요 (https://www.microsoft.com/en-us/download/confirmation.aspx?id=40773)
# MS build toold 설치 : https://visualstudio.microsoft.com/ko/downloads/?q=build
# !pip install jpype1  # 
# !pip install konlpy, customized_konlpy

## 자연어처리
### [자연 언어 처리 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%9E%90%EC%97%B0_%EC%96%B8%EC%96%B4_%EC%B2%98%EB%A6%AC)
* 인간이 발화하는 언어를 기계적으로 분석해서 자연어와 같이 구조화되지 않은 비정형 텍스트로부터 새로운 지식을 발견하는 과정

NLP는?
NLP(자연어처리)는 텍스트 문제에 접근하기 위한 기술집합이다.

자연어처리(natural language processing, NLP) 분야는 인공지능의 큰 줄기 중에 하나입니다. 특히, 컴퓨터에게 사람이 사용하는 언어를 처리하고 이해하도록 함으로써, 사람과 컴퓨터 사이의 매개체 또는 인터페이스 역할을 할 수 있습니다. 따라서 컴퓨터 공학 뿐만 아니라, 언어학과 같은 다른 학문과의 융합적인 요소도 갖고 있습니다.

따라서, 이러한 자연어처리의 세부적인 부분들이 합쳐져 최종적인 목표는 사람의 언어를 이해하여 컴퓨터로 하여금 여러가지 문제를 수행할 수 있도록 하는 것입니다. 컴퓨터는 이제 우리와 뗄 수 없는 존재가 되었고, 그러므로 이미 실제로 자연어처리는 우리의 일상에 가장 깊숙히 들어와 있는 분야이기도 합니다. 자연어처리 기술에 의해서 수행되는 대표적인 문제 또는 응용분야들은 다음과 같습니다.





### 자연어처리(NLP)와 관련 된 캐글 경진대회
* [Sentiment Analysis on Movie Reviews | Kaggle](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews)
* [Toxic Comment Classification Challenge | Kaggle](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)
* [Spooky Author Identification | Kaggle](https://www.kaggle.com/c/spooky-author-identification)

### 데이터 정제 Data Cleaning and Text Preprocessing
기계가 텍스트를 이해할 수 있도록 텍스트를 정제해 준다.

신호와 소음을 구분한다. 아웃라이어데이터로 인한 오버피팅을 방지한다.

1. BeautifulSoup(뷰티풀숩)을 통해 HTML 태그를 제거
2. 정규표현식으로 알파벳 이외의 문자를 공백으로 치환
3. NLTK 데이터를 사용해 불용어(Stopword)를 제거
4. 어간추출(스테밍 Stemming)과 음소표기법(Lemmatizing)의 개념을 이해하고 SnowballStemmer를 통해 어간을 추출


### 텍스트 데이터 전처리 이해하기

(출처 : [트위터 한국어 형태소 분석기](https://github.com/twitter/twitter-korean-text))

**정규화 normalization (입니닼ㅋㅋ -> 입니다 ㅋㅋ, 샤릉해 -> 사랑해)**

* 한국어를 처리하는 예시입니닼ㅋㅋㅋㅋㅋ -> 한국어를 처리하는 예시입니다 ㅋㅋ

**토큰화 tokenization**

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하는Verb, 예시Noun, 입Adjective, 니다Eomi ㅋㅋKoreanParticle

**어근화 stemming (입니다 -> 이다)**

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하다Verb, 예시Noun, 이다Adjective, ㅋㅋKoreanParticle


**어구 추출 phrase extraction** 

* 한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어, 처리, 예시, 처리하는 예시

Introductory Presentation: [Google Slides](https://docs.google.com/presentation/d/10CZj8ry03oCk_Jqw879HFELzOLjJZ0EOi4KJbtRSIeU/)

In [5]:
from konlpy.tag import Kkma,Okt,Hannanum
from konlpy.utils import pprint
from pprint import pprint
import ckonlpy
from collections import Counter
import matplotlib.pyplot as plt

%matplotlib inline

# Read data

In [1]:
# 북미정상회담 데이터 
with open('./data/pyongyang_fin.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
texts[:1000]

"존경하는 국민여러분. 성원해 주신 덕분에 평양에 잘 다녀왔습니다. \n\n국민 여러분께서 보셨듯이 정상회담에서 좋은 합의를 이뤘고, 최상의 환대를 받았습니다. 무엇보다 3일동안 김정은 위원장과 여러차례 만나 긴 시간 많은 대화를 허심탄회하게 나눌 수 있었던 것에 큰 의미를 두고 싶습니다.\n\n남북관계를 크게 진전시키고 두 정상 간의 신뢰 구축에도 큰 도움이 된 방문이었다고 평가하고 싶습니다.\n\n북측에서는 짧은 준비기간에도 불구하고 우리 대표단을 정성을 다해 맞아 주었습니다.\n\n오고 가는 동안 공항과 길가에서 열렬하게 환영해주고 환송해준 평양 시민들께 각별한 인사를 드리지 않을 수 없습니다.\n\n백두산에 오가는 동안 삼지연공항에서 따뜻하게 맞아주고 배웅해 준 지역 주민들께도 감사드립니다.\n\n저는 5월1일 경기장에서 열린 대규모 집단체조와 공연에서 15만 평양 시민들에게 대한민국 대통령으로써 사상 최초로 연설을 하는 기회를 가졌습니다.\n\n그들은 한반도를 영구히 핵무기와 핵위협이 없는 평화의 터전으로 만들어야 한다는 저의 연설에 대해 열렬한 박수를 보내줬습니다.\n\n존경하는 국민 여러분.\n\n지난 3일간 저는 김정은 위원장과 비핵화와 북미 대화에 대해서도 많은 대화를 나누었습니다. 첫날 회담에서도 대부분의 시간을 비핵화를 논의하는데 사용했습니다.\n\n김정은 위원장은 확고한 비핵화 의지를 거듭, 거듭 확약했습니다.\n\n가능한 한 빠른 시기에 완전한 비핵화를 끝내고 경제발전에 집중하고 싶다는 희망을 밝혔습니다.\n\n다만 북미정상회담에서 합의한 4개 합의사항이 함께 이행돼야 하므로 미국이 그 정신에 따라 상응하는 조치를 취해준다면, 영변 핵시설의 영구적 폐기를 포함한 추가적인 비핵화 조치를 계속 실행해나갈 용의가 있음을 표명했습니다.\n\n그리고 그 의지를 다시 한 번 분명하게 밝히는 차원에서 우선 동창리 미사일 엔진 시험장과 미사일 발사대를 유관국 전문가들의 참관 하에 영구적으로 폐기할 것을 확약했습니다.\n\n북한이 평양공동선언에서 사용한

In [ ]:
# excel , csv
df = pd.read_excel('./data/xx.xlsx')
# df = pd.read_csv('./data/xx.xlsx')

# hive, db
# conn = pyodbc.connect('DSN_B02_HIVE_ODBC;UID=da2_master;PWD=xxxx;charaset=utf8',autocommit=True)
# sql = "select * from xxx"
# df = pd.read_sql(sql, conn)

# pyspark
# df = hc.sql('select * from xxx')

# EDA

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.hist(bins=10)

# Text Preprocessing

In [ ]:
df_text = df['text'].copy()

In [2]:
taggers = [('kkma',Kkma()),
           ('twitter', Okt()),
           ('hannanum', Hannanum())    
           # mecab은 window에서 지원안됨
]
taggers

NameError: name 'Kkma' is not defined

In [ ]:
pprint(taggers[0][1].pos('테스트 문장'))

In [ ]:
pprint(taggers[1][1].pos('테스트 문장'))

In [ ]:
pprint(taggers[2][1].pos('테스트 문장'))

In [ ]:
pprint(taggers[2][1].tagset))

# pos tagging & word count

In [ ]:
sents = df_text.values.tolist()

In [ ]:
import time

token = []

for name, tagger in taggers:
    
    process_time = time.time()
    tokens.append([pos for sent in sents for pos in tagger.pos(sent)])
    process_time = time.time() - process_time
    
    print('tagger name = {}, {:.3} secs'.format(name, process_time))

In [ ]:
len(tokens)

In [ ]:
counter = Counter(token[0])
count = {word:freq for word, freq in counter.items() if (freq>=4) and (word[1:2] == 'NN')}
pprint(sorted(counter.items(), key=lambda x:x[1], reverse=True))

In [ ]:
for (name, _), _tokens in zip(taggers, tokens):
    print('pos {}'.format(name))
    counter = Counter(_tokens)
    counter = {word:freq for word, freq in counter.items() if (freq>=4) and (word[1:1] == 'N')}
    pprint(sorted(counter.items(), key=lambda x:x[1], reverse=True))

In [ ]:

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Honeycrisp.jpg/1280px-Honeycrisp.jpg' width=200>
* 이미지 출처 https://en.wikipedia.org/wiki/Apple
